Read Mesh

In [3]:
from lnas import LnasFormat
import pathlib

meshes_path = pathlib.Path("./fixtures/lnas")

mesh = LnasFormat.from_file(meshes_path / "G100.lnas")
mesh.geometry_from_surface("m_000").export_stl(meshes_path / "G100.m_000.stl")
negative_mesh = LnasFormat.from_file(meshes_path / "G100_negative.lnas")
negative_mesh.geometry_from_surface("m_000").export_stl(meshes_path / "G100_negative.m_000.stl")

Offset body

In [6]:
import numpy as np

NORMAL_OFFSET = -0.125

geometry_negative = negative_mesh.geometry
geometry_negative.vertices = (
    geometry_negative.vertices + geometry_negative.vertices_normals * NORMAL_OFFSET
)

print(np.max(mesh.geometry.triangles), len(mesh.geometry.vertices))
print(np.max(negative_mesh.geometry.triangles), len(negative_mesh.geometry.vertices))


def combine_lnas(
    lnas_fmts: list[LnasFormat], surfaces_suffixes: list[str] | None = None
) -> LnasFormat:
    if len(lnas_fmts) == 0:
        raise ValueError("No LNAS to combine")
    if surfaces_suffixes is not None:
        if len(surfaces_suffixes) < len(lnas_fmts):
            raise ValueError("Less surfaces suffixes than required")
    merged_geo = lnas_fmts[0].geometry.copy()
    surfaces = {}

    suffix = surfaces_suffixes[0] if surfaces_suffixes is not None else ""
    for s, arr in lnas_fmts[0].surfaces.items():
        surfaces[s + suffix] = arr.copy()

    for i, lnas_fmt in enumerate(lnas_fmts):
        if i == 0:
            # Already added
            continue
        n_verts, n_tris = len(merged_geo.vertices), len(merged_geo.triangles)

        verts_add = lnas_fmt.geometry.vertices.copy()
        merged_geo.vertices = np.concatenate((merged_geo.vertices.copy(), verts_add), axis=0)

        tri_add = lnas_fmt.geometry.triangles + n_verts
        merged_geo.triangles = np.concatenate((merged_geo.triangles.copy(), tri_add), axis=0)

        suffix = surfaces_suffixes[i] if surfaces_suffixes is not None else ""
        for s, arr in lnas_fmt.surfaces.items():
            key = s + suffix
            if key in surfaces:
                raise KeyError(
                    f"Surface {s} is already in the list of surfaces, provide a suffix for it"
                )
            surfaces[key] = arr + n_tris

    merged_geo._full_update()

    return LnasFormat(version=lnas_fmts[0].version, geometry=merged_geo, surfaces=surfaces)


merged_lnas = combine_lnas([mesh, negative_mesh], ["", "_inside"])

21730 21731
1373 1374


In [7]:
new_fmt = merged_lnas

filename = meshes_path / "G100.merged.lnas"
export_path = meshes_path / "G100.merged.1.stl"

new_fmt.to_file(filename)

for s in new_fmt.surfaces:
    geometry = new_fmt.geometry_from_surface(s)
    geometry.export_stl(meshes_path / f"G100.merged.{s}.stl")

    # geometry.export_stl(meshes_path / "m_000_inside.stl")
    print(s)
    print(geometry.vertices[:, 0].min(), geometry.vertices[:, 0].max())
    print(geometry.vertices[:, 1].min(), geometry.vertices[:, 1].max())
    print(geometry.vertices[:, 2].min(), geometry.vertices[:, 2].max())
# mesh.export_stl(export_path)

w_xp_001
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
w_xm_001
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
b_xp_004
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
b_xm_000
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
w_ym_000
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
002
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
b_xm_001
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
extr_001_0
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
001
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
m_001
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
m_003
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
t_000
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
b_xp_000
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
lant
-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
b_xm_003
-427.4785 -124.554756
-245.32094 159.66632
837

In [24]:
mesh = LnasFormat.from_file(meshes_path / "G100.merged.lnas")

# mesh.geometry_from_surface("m_000_inside").export_stl(meshes_path / "m_000_inside.stl")
geometry = mesh.geometry_from_surface("m_000_inside")
export_path = meshes_path / "G100.merged.2.stl"
geometry.export_stl(export_path)  # type: ignore
print(geometry.vertices[:, 0].min(), geometry.vertices[:, 0].max())
print(geometry.vertices[:, 1].min(), geometry.vertices[:, 1].max())
print(geometry.vertices[:, 2].min(), geometry.vertices[:, 2].max())

-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
